##1: Análisis de Sentimiento con FinBERT

In [3]:
!pip install transformers datasets torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.0 MB/s eta 0:00:00


In [4]:
!pip install pymupdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 12.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import fitz  # PyMuPDF
from transformers import pipeline as hf_pipeline
from google.colab import files

In [13]:
print("Tarea 1: Subir 4 archivos txt del Financial Phrasebank")

from google.colab import files
import pandas as pd

uploaded = files.upload()

Tarea 1: Subir 4 archivos txt del Financial Phrasebank


Saving Sentences_50Agree.txt to Sentences_50Agree (1).txt
Saving Sentences_66Agree.txt to Sentences_66Agree (1).txt
Saving Sentences_75Agree.txt to Sentences_75Agree (1).txt
Saving Sentences_AllAgree.txt to Sentences_AllAgree (1).txt


In [17]:
dfs = []

for filename in uploaded.keys():
    print(f"Leyendo archivo: {filename}")
    with open(filename, "r", encoding="latin1") as f:
        lines = f.readlines()

    data = []
    for line in lines:
        line = line.strip()
        if "@positive" in line:
            sentence, sentiment = line.rsplit("@positive", 1)
            data.append([sentence.strip(), "positive"])
        elif "@negative" in line:
            sentence, sentiment = line.rsplit("@negative", 1)
            data.append([sentence.strip(), "negative"])
        elif "@neutral" in line:
            sentence, sentiment = line.rsplit("@neutral", 1)
            data.append([sentence.strip(), "neutral"])

    df = pd.DataFrame(data, columns=["sentence", "sentiment"])
    dfs.append(df)

# Unimos todos los DataFrames
full_df = pd.concat(dfs, ignore_index=True)

print("\n✅ Ejemplo de frases y etiquetas:")
print(full_df.head())

print("\n📊 Distribución de etiquetas:")
print(full_df["sentiment"].value_counts())

Leyendo archivo: Sentences_50Agree (1).txt
Leyendo archivo: Sentences_66Agree (1).txt
Leyendo archivo: Sentences_75Agree (1).txt
Leyendo archivo: Sentences_AllAgree (1).txt

✅ Ejemplo de frases y etiquetas:
                                            sentence sentiment
0  According to Gran , the company has no plans t...   neutral
1  Technopolis plans to develop in stages an area...   neutral
2  The international electronic industry company ...  negative
3  With the new production plant the company woul...  positive
4  According to the company 's updated strategy f...  positive

📊 Distribución de etiquetas:
sentiment
neutral     8951
positive    3988
negative    1841
Name: count, dtype: int64


In [18]:
#Cargar el modelo FinBERT
from transformers import pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Dispositivo utilizado: {device}")

print("⏳ Cargando modelo FinBERT...")
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0 if device == "cuda" else -1)

⚙️ Dispositivo utilizado: cpu
⏳ Cargando modelo FinBERT...


Device set to use cpu


In [19]:
#Aplicar FinBERT a una muestra de frases
if full_df.empty:
    print("⚠️ El DataFrame está vacío. Revisa si los archivos tienen frases.")
else:
    sample_texts = full_df["sentence"].sample(10, random_state=1).tolist()

    print("\n🔍 Predicciones del modelo FinBERT:")
    results = finbert(sample_texts)

    for i, res in enumerate(results):
        print(f"\n📌 Frase: {sample_texts[i]}")
        print(f"🔎 Predicción: {res['label']} (Confianza: {res['score']:.2f})")


🔍 Predicciones del modelo FinBERT:

📌 Frase: According to Viking Line 's Managing Director , Nils-Erik Eklund , the company 's Board of Directors is very satisfied with Viking Line 's performance .
🔎 Predicción: positive (Confianza: 0.95)

📌 Frase: Finnish steel maker Rautaruukki Oyj ( HEL : RTRKS ) , or Ruukki , said today its net loss contracted to EUR 49 million ( USD 68.2 m ) for the first nine months of 2010 from EUR 229 million for the same period a year ago .
🔎 Predicción: positive (Confianza: 0.90)

📌 Frase: There have not been previous share subscriptions with 2004 stock options .
🔎 Predicción: neutral (Confianza: 0.95)

📌 Frase: hu will offer a further discount of between 25 % and 50 % on selected books .
🔎 Predicción: neutral (Confianza: 0.89)

📌 Frase: A total of 15,000 new Citycon shares with a nominal value of EUR 1.35 per share were subscribed between 17 and 23 March 2006 exercising the A-B-C options based on the company 's stock option plan 1999 .
🔎 Predicción: neutral

##2: Resumen automático de un contrato en PDF

In [6]:
print("📂 Sube el contrato en formato PDF")

uploaded_pdf = files.upload()

📂 Sube el contrato en formato PDF


Saving Contrato_ejercicio.pdf to Contrato_ejercicio.pdf


In [7]:
import fitz  # PyMuPDF

# Obtener nombre del archivo PDF
pdf_filename = list(uploaded_pdf.keys())[0]

# Leer contenido del PDF
with fitz.open(pdf_filename) as doc:
    text = ""
    for page in doc:
        text += page.get_text()

print("\n📄 Texto extraído del contrato:")
print(text[:1000])  # mostrar los primeros 1000 caracteres


📄 Texto extraído del contrato:
ARRENDAMIENTO DE LOCAL COMERCIAL 
 
 
Conste por la suscripción del presente contrato de Arrendamiento o Alquiler del Local 
Comercial que celebran, de una parte ZAMORA QUISPE, MARÍA ELENA, 
identificada con DNI N.º 00000001, con domicilio real en Calle Los Cipreses N.º 123, 
Urb. Jardines del Sol, Distrito de San Martín de Porres, a quien en adelante se le 
denominará LA DUEÑA o ARRENDADORA; y de la otra parte, el señor 
VELÁSQUEZ MENDOZA, JULIO, identificado con DNI N.º 00000002, con domicilio 
en Mz. A Lt. 15, Conjunto Residencial Santa Clara, Distrito de Los Olivos, a quien 
en adelante se le denominará EL ARRENDATARIO o INQUILINO, bajo los 
siguientes términos: 
 
ANTECEDENTES 
 
 
PRIMERA: LA ARRENDADORA declara ser propietaria del inmueble ubicado en la 
Av. Los Laureles N.º 450, Mz. B Lt. 20, Urb. Villa Central, Distrito de 
Independencia, entre las avenidas Los Laureles y Alameda del Norte. 
 
 
SEGUNDA: LA ARRENDADORA deja constancia que el loc

In [8]:
from transformers import pipeline

# Cargar modelo
print("⏳ Cargando modelo de resumen (facebook/bart-large-cnn)...")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Dividir texto en partes (1000 caracteres como máximo para evitar el límite de tokens)
def split_text(text, max_length=1000):
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]

# Crear los fragmentos
text_chunks = split_text(text)

# Mostrar cuántos fragmentos se procesarán
print(f"\n📄 Número de fragmentos a resumir: {len(text_chunks)}")

# Generar resumen para cada fragmento
summaries = []
for i, chunk in enumerate(text_chunks):
    print(f"🔄 Resumiendo parte {i + 1} de {len(text_chunks)}...")
    summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# Unir los resúmenes
final_summary = ' '.join(summaries)

print("\n✅ Resumen completo generado:")
print(final_summary)

⏳ Cargando modelo de resumen (facebook/bart-large-cnn)...


Device set to use cuda:0



📄 Número de fragmentos a resumir: 11
🔄 Resumiendo parte 1 de 11...
🔄 Resumiendo parte 2 de 11...
🔄 Resumiendo parte 3 de 11...
🔄 Resumiendo parte 4 de 11...
🔄 Resumiendo parte 5 de 11...
🔄 Resumiendo parte 6 de 11...
🔄 Resumiendo parte 7 de 11...
🔄 Resumiendo parte 8 de 11...
🔄 Resumiendo parte 9 de 11...
🔄 Resumiendo parte 10 de 11...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🔄 Resumiendo parte 11 de 11...

✅ Resumen completo generado:
Arrendamiento o Alquiler del Local                 Comercial que celebran, de una parte ZAMORA QUISPE, MARÍA ELENA, y de la otra parte, el señor VELÁSQUEZ MENDOZA, JULIO. La Alameda del Norte N.º 451, en buen estado de conservación y habitabilidad, con piso de cerámica tipo porcelanato. El presente contrato, LA ARRENDADORA se obliga a ceder el uso del local comercial, concerniente como local comerscial, a favor of EL  ARRENDATARIO, a título de alquiler. El monto de la renta pactada en la cláusula siguiente. La renta será pago por mensualidades, realizándose el abono 11 de cada mes. El comprobante de pago deberá ser enviado por correo electrónico o entregado a LA ARRENDADORA dentro of las 24 horas posteriores a cada depósito. El Código Civil está obligado a pagar puntualmente el monto de todos los servicios inherentes al bien tales como agua y luz eléctrica, parques y jardines with el impuesto del Patrimonio. Así mismo, EL ARR

#Resumen
Arrendamiento o Alquiler del Local                 Comercial que celebran, de una parte ZAMORA QUISPE, MARÍA ELENA, y de la otra parte, el señor VELÁSQUEZ MENDOZA, JULIO. La Alameda del Norte N.º 451, en buen estado de conservación y habitabilidad, con piso de cerámica tipo porcelanato. El presente contrato, LA ARRENDADORA se obliga a ceder el uso del local comercial, concerniente como local comerscial, a favor of EL  ARRENDATARIO, a título de alquiler. El monto de la renta pactada en la cláusula siguiente. La renta será pago por mensualidades, realizándose el abono 11 de cada mes. El comprobante de pago deberá ser enviado por correo electrónico o entregado a LA ARRENDADORA dentro of las 24 horas posteriores a cada depósito. El Código Civil está obligado a pagar puntualmente el monto de todos los servicios inherentes al bien tales como agua y luz eléctrica, parques y jardines with el impuesto del Patrimonio. Así mismo, EL ARRENDATARIO esta obligado toda la entrega de las llaves  del alquiler. El arrendatario se obliga a destinar el bien exclusivamente para FARMACIA. El bien deberá cruzar previo aviso por escrito, con una anticipación no de 7 días. EL ARRENDATARIO no podrá ceder a terceros el bien material bajo ningún título, ni subarrendar, total o parcial. En caso de incumplimiento de lo estipulado en la cláusula séptima, EL ARRENTARARIO deberá pagar en calidad de penalidad a 10 dólares. El mencionado depósito en garantía le será devuelto a EL ARRENDATARIO sin  vencimiento del contrato, una vez verificado el estado de conservación. El pago por mes  purposefullyadelantado de Setiembre del 2024 por la suma de S/ 1400 MIL CUATROCIENTOS  purposefullySOLES. Son causales de resolución del contrato el incumplimiento las causales establecidas en el art. 1697° del Código Civil siempre que no se encuentren comprendidas su tratamiento especial en el presente contrato. De conformidad al art. 5° de la Ley 30201. LA ARRENDADORA y EL ARR ENDATARIO declaran expresamente y de manera ishlyindubitable, que se someten a los efectos that contiene de la Ley 30933. En ese sentido, se produce el vencimiento del plazo del presente contrato de arrendamiento. VIGÉSIMO CUARTA: En lo previsto de las partes del presente contrato ambas partes  someten a lo establecido by las normas del Código Civil. Así mismo LA ARRENDADORA o dueña del bien, deja  purposefully constancia que puede ampararse. IO.IO.io is a web site that allows users to share photos and other information. The site is open to the public and can be accessed by anyone by clicking here. Click here to visit.
